In [1]:
import os
import glob
import rasterio
import shutil

In [ ]:
# 找一些具有代表性的crop来比较几种方法的效果

In [2]:
base_dir = '/scratch2/ziyliu/pro_data/sate_dataset_V2/test_all'
comparedir = '/scratch2/ziyliu/Inpainting/final_comparison/'

In [3]:
mask_filenames = sorted(list(glob.glob(os.path.join(base_dir, '**', '*mask*.tif'), recursive=True)))
img_filenames = [fname.rsplit('_mask', 1)[0] + '.tif' for fname in mask_filenames]

In [4]:
print(len(mask_filenames), len(img_filenames))

3589 3589


In [5]:
for img_fname, mask_fname in zip(img_filenames, mask_filenames):
    with rasterio.open(img_fname) as src_img:
        img_array = src_img.read(1).astype('float32')
        img_nodata = img_array == 2 ** 16 - 1
    if img_nodata.sum() == 0:
        shutil.copy(img_fname, os.path.join(comparedir, os.path.basename(img_fname)))
        shutil.copy(mask_fname, os.path.join(comparedir, os.path.basename(mask_fname)))

In [3]:
import numpy as np
base_dir = '/scratch2/ziyliu/Inpainting/final_comparison/'
img_filenames = sorted(list(glob.glob(os.path.join(base_dir, '**', '*_mask*.tif'), recursive=True)))

for img_fname in img_filenames:
    with rasterio.open(img_fname) as src_img:
        profile = src_img.profile
    mask_fname = img_fname.replace('.tif', '0.4.tif')
    mask_array = np.ones((256, 256), dtype='uint8')
    mask_array[:, 256//5*2:] = 0    

    with rasterio.open(mask_fname, 'w', **profile) as dst:
        dst.write(mask_array, 1)